## Richa Motwani
2019120040

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/data.csv')

In [ ]:
df.head()

,genre,company,origin,duration,time,place,enjoy movie
0,romantic,yes,bollywood,short,morning,home,yes
1,action,yes,bollywood,short,morning,theatre,yes
2,thriller,no,hollywood,short,evening,theatre,no
3,comedy,yes,bollywood,short,morning,home,yes
4,comedy,yes,hollywood,short,morning,home,yes


In [ ]:
def calc_total_entropy(train_data, label, class_list):
    total_row = train_data.shape[0] #the total size of the dataset
    total_entr = 0
    
    for c in class_list: #for each class in the label
        total_class_count = train_data[train_data[label] == c].shape[0] #number of the class
        total_class_entr = - (total_class_count/total_row)*np.log2(total_class_count/total_row) #entropy of the class
        total_entr += total_class_entr #adding the class entropy to the total entropy of the dataset
        
    return total_entr

In [ ]:
def calc_entropy(feature_value_data, label, class_list):
    class_count = feature_value_data.shape[0]
    entropy = 0
    
    for c in class_list:
        label_class_count = feature_value_data[feature_value_data[label] == c].shape[0] #row count of class c 
        entropy_class = 0
        if label_class_count != 0:
            probability_class = label_class_count/class_count #probability of the class
            entropy_class = - probability_class * np.log2(probability_class)  #entropy
        entropy += entropy_class
    return entropy

In [ ]:
def calc_info_gain(feature_name, train_data, label, class_list):
    feature_value_list = train_data[feature_name].unique() #unqiue values of the feature
    total_row = train_data.shape[0]
    feature_info = 0.0
    
    for feature_value in feature_value_list:
        feature_value_data = train_data[train_data[feature_name] == feature_value] #filtering rows with that feature_value
        feature_value_count = feature_value_data.shape[0]
        feature_value_entropy = calc_entropy(feature_value_data, label, class_list) #calculcating entropy for the feature value
        feature_value_probability = feature_value_count/total_row
        feature_info += feature_value_probability * feature_value_entropy #calculating information of the feature value
        print('Feature value is {}, Entropy is {}'.format(feature_value, feature_value_entropy))

    return calc_total_entropy(train_data, label, class_list) - feature_info #calculating information gain by subtracting view raw


In [ ]:
def find_most_informative_feature(train_data, label, class_list):
    feature_list = train_data.columns.drop(label) #finding the feature names in the dataset
                                            #N.B. label is not a feature, so dropping it
    max_info_gain = -1
    max_info_feature = None
    print("---------------")
    for feature in feature_list:  #for each feature in the dataset
        print('Feature is {}'.format(feature))
        feature_info_gain = calc_info_gain(feature, train_data, label, class_list)
        print('{} Gain is {}'.format(feature,feature_info_gain))
        if max_info_gain < feature_info_gain: #selecting feature name with highest information gain
            max_info_gain = feature_info_gain
            max_info_feature = feature
    print('Root Node will be {}, having highest gain of {}'.format(max_info_feature,max_info_gain))   
    return max_info_feature, max_info_gain

In [ ]:
def generate_sub_tree(feature_name, train_data, label, class_list):
    feature_value_count_dict = train_data[feature_name].value_counts(sort=False) #dictionary of the count of unqiue feature value
    tree = {} #sub tree or node
    
    for feature_value, count in feature_value_count_dict.iteritems():
        feature_value_data = train_data[train_data[feature_name] == feature_value] #dataset with only feature_name = feature_value
        
        assigned_to_node = False #flag for tracking feature_value is pure class or not
        for c in class_list: #for each class
            class_count = feature_value_data[feature_value_data[label] == c].shape[0] #count of class c

            if class_count == count: #count of feature_value = count of class (pure class)
                tree[feature_value] = c #adding node to the tree
                train_data = train_data[train_data[feature_name] != feature_value] #removing rows with feature_value
                assigned_to_node = True
        if not assigned_to_node: #not pure class
            tree[feature_value] = "?" #should extend the node, so the branch is marked with ?
            
    return tree, train_data

In [ ]:
def make_tree(root, prev_feature_value, train_data, label, class_list):
    if train_data.shape[0] != 0: #if dataset becomes enpty after updating
        max_info_feature, max_info_gain = find_most_informative_feature(train_data, label, class_list) #most informative feature
        tree, train_data = generate_sub_tree(max_info_feature, train_data, label, class_list) #getting tree node and updated dataset
        next_root = None
        
        if prev_feature_value != None: #add to intermediate node of the tree
            root[prev_feature_value] = dict()
            root[prev_feature_value][max_info_feature] = tree
            next_root = root[prev_feature_value][max_info_feature]
        else: #add to root of the tree
            root[max_info_feature] = tree
            next_root = root[max_info_feature]
        
        for node, branch in list(next_root.items()): #iterating the tree node
            if branch == "?": #if it is expandable
                feature_value_data = train_data[train_data[max_info_feature] == node] #using the updated dataset
                make_tree(next_root, node, feature_value_data, label, class_list) #recursive call with updated dataset

In [ ]:
def id3(train_data_m, label):
    train_data = train_data_m.copy() #getting a copy of the dataset
    tree = {} #tree which will be updated
    class_list = train_data[label].unique() #getting unqiue classes of the label
    make_tree(tree, None, train_data_m, label, class_list) #start calling recursion
    return tree

In [ ]:
ree = id3(df, 'enjoy movie')

---------------
Feature is genre
Feature value is romantic, Entropy is 0.9709505944546686
Feature value is action, Entropy is 0.9182958340544896
Feature value is thriller, Entropy is 0.9709505944546686
Feature value is comedy, Entropy is 0.0
genre Gain is 0.22582953145948093
Feature is company
Feature value is yes, Entropy is 0.4394969869215134
Feature value is no, Entropy is 0.863120568566631
company Gain is 0.31405634315987485
Feature is origin
Feature value is bollywood, Entropy is 0.8112781244591328
Feature value is hollywood, Entropy is 0.9709505944546686
Feature value is high, Entropy is 0.0
origin Gain is 0.10773525262210415
Feature is duration
Feature value is short, Entropy is 0.6193821946787638
Feature value is long, Entropy is 0.7219280948873623
duration Gain is 0.27042866709555957
Feature is time
Feature value is morning, Entropy is 0.0
Feature value is evening, Entropy is 0.5916727785823275
time Gain is 0.6882008646058067
Feature is place
Feature value is home, Entropy is 

In [ ]:
ree

{'time': {'morning': 'yes',
  'evening': {'genre': {'thriller': 'no',
    'romantic': 'no',
    'comedy': 'yes',
    'action': 'no'}}}}

In [ ]:
import pydot

def draw(parent_name, child_name):
    edge = pydot.Edge(parent_name, child_name)
    graph.add_edge(edge)

def visit(node, parent=None):
    for k,v in node.items():
        if isinstance(v, dict):
            # We start with the root node whose parent is None
            # we don't want to graph the None node
            if parent:
                draw(parent, k)
            visit(v, k)
        else:
            draw(parent, k)
            # drawing the label using a distinct name
            draw(k, k+'_'+v)

graph = pydot.Dot(graph_type='graph')
visit(ree)
graph.write_png('example1_graph.png')

In [ ]:
graph